In [2]:
import os
# import requests
# import time
# import csv
# import http.client
import json
import pandas as pd
import numpy as np



# import geopandas as gpd
# import pyogrio
# import fiona

# from geopy.exc import GeocoderTimedOut
# from geopy.geocoders import Nominatim

import plotly.express as px
import plotly.graph_objects as go

# import country_converter as coco

pd.set_option('display.max_columns', 70)

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "vscode"


In [3]:
PWD = os.getcwd()
DATA_PATH = PWD + "/data/"

WALL_DATA_PATH = DATA_PATH + "wall-data.js"
CRIME_DATA_PATH = DATA_PATH + "crime-data.xlsx"
CITY_TO_POSTCODE_DATA_PATH = DATA_PATH + "city-state-postcode.csv"
POPULATION_TO_POSTCODE_DATA_PATH = DATA_PATH + "population-postcode.csv"
POLYGON_TO_POSTCODE_DATA_PATH = DATA_PATH + "postcode-polygon.geojson"


## Berlin walk data downloaded from All Trails

## Berlin districts downloaded from
https://daten.odis-berlin.de/en/dataset/bezirksgrenzen/
https://daten.odis-berlin.de/de/dataset/plz/
https://www.suche-postleitzahl.org/downloads

Tutorial: https://juanitorduz.github.io/germany_plots/

## Crime data downloaded as .xlsx from Berlin Police department
- https://www.berlin.de/polizei/

## !!! Load the Berlin Postcode Polygon data !!!

In [50]:
with open(POLYGON_TO_POSTCODE_DATA_PATH) as f:
    js_data = f.read()

# Extract the JSON part (assuming it's properly formatted)
# Here, we extract the part after 'export default '
json_data = js_data[js_data.find('{'):]  # Get everything from the first '{'

# Load the JSON data
berlin_district_polygon_geojson = json.loads(json_data)

# Extract coordinates from the GeoJSON
str(berlin_district_polygon_geojson)[0:300]

"{'type': 'FeatureCollection', 'name': 'plz', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'plz': '10115'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[13.365859811970504, 52.53565689704142], [13.36828617431849"

In [45]:
df_polygon_postcode = pd.DataFrame(columns=['plz', 'lat', 'lon'])
df_rows = []

lat1, lon1 = [], []

for i, val in enumerate(berlin_district_polygon_geojson['features']):

    feature = berlin_district_polygon_geojson['features'][i]

    postcode = feature['properties']['plz']
    polygon_type = feature['geometry']['type']

    if polygon_type == 'Polygon':
        coordinates = np.array(feature['geometry']['coordinates'])
        coordinates = np.squeeze(coordinates)
        temp_lon, temp_lat = map(list, zip(*coordinates))

        lat1 +=  temp_lat
        lon1 += temp_lon

        new_row = {'plz': postcode, 'lat':temp_lat, 'lon':temp_lon}
        df_rows.append(new_row)
    else:
         print(f"MultiPolygoon postcode (probably an island) = {i, feature['properties']['plz']}")



df_polygon_postcode = pd.concat([df_polygon_postcode, pd.DataFrame(df_rows)], ignore_index=True)


df_polygon_postcode.head()


MultiPolygoon postcode (probably an island) = (67, '12059')
MultiPolygoon postcode (probably an island) = (89, '12305')
MultiPolygoon postcode (probably an island) = (100, '12437')
MultiPolygoon postcode (probably an island) = (109, '12559')
MultiPolygoon postcode (probably an island) = (114, '12623')
MultiPolygoon postcode (probably an island) = (118, '12681')
MultiPolygoon postcode (probably an island) = (131, '13125')


,plz,lat,lon
0,10115,"[52.53565689704142, 52.53329426945583, 52.5332...","[13.365859811970504, 13.368286174318495, 13.36..."
1,10117,"[52.52780270094086, 52.527699530631956, 52.527...","[13.373740754185107, 13.373822387458334, 13.37..."
2,10119,"[52.527012865270976, 52.52630645770313, 52.526...","[13.399022108201152, 13.40133607102495, 13.401..."
3,10178,"[52.527012865270976, 52.52679311454601, 52.526...","[13.399022108201152, 13.39877276099848, 13.398..."
4,10179,"[52.51216582078395, 52.51186033902778, 52.5117...","[13.403053660264343, 13.402610386070572, 13.40..."


## !!! Load the Berlin Wall data !!!

In [56]:
with open(WALL_DATA_PATH) as f:
    js_data = f.read()

# Extract the JSON part (assuming it's properly formatted)
# Here, we extract the part after 'export default '
json_data = js_data[js_data.find('{'):]  # Get everything from the first '{'

# Load the JSON data
berlin_wall_geojson = json.loads(json_data)

# Extract coordinates from the GeoJSON
coordinates = berlin_wall_geojson['features'][0]['geometry']['coordinates'][0]

lon, lat, height = map(list, zip(*coordinates))

lon.append(lon[0])
lat.append(lat[0])
height.append(height[0])



In [57]:
# Earth radius in kilometers
R = 6371.0

def haversine_np(lat, lon):
    # Convert latitude and longitude from degrees to radians
    lat = np.radians(lat)
    lon = np.radians(lon)
    
    # Compute the differences between consecutive points
    dlat = lat[1:] - lat[:-1]
    dlon = lon[1:] - lon[:-1]
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat[:-1]) * np.cos(lat[1:]) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distances = R * c
    
    # Insert zero at the start to represent distance from the first point
    distances = np.insert(distances, 0, 0.0)
    
    return distances

# Example latitude and longitude lists
latitude = np.array([...])  # Replace with your latitude values
longitude = np.array([...])  # Replace with your longitude values

# Calculate distances
dist = haversine_np(lat, lon)
tot_dist = np.cumsum(dist)

height = np.array(height)
tot_height = np.cumsum(height)


In [58]:
berlinwall_df = pd.DataFrame({'lat':lat, 'lon':lon, 'height':height, 'tot_height':tot_height, 'dist':dist, 'tot_dist':tot_dist})


# Create a scatter map with plotly
fig = px.scatter_mapbox(
    berlinwall_df,
    lat="lat",
    lon="lon",
    title="Berlin Wall",
    zoom=9,
    mapbox_style="carto-positron"
)

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={"x": 0.5, "xanchor": "center"},
    height = 400,
    width = 400
)

fig.show()


In [53]:
# px.line(berlinwall_df, x="tot_dist", y="height")

In [54]:
# px.line(berlinwall_df, x="tot_dist", y="tot_height")

## !!! Load the crime data !!!

The crime burden of the districts and district regions is shown on the basis of the frequency number (HZ). The HZ is a crime quotient that relates the number of cases that have become known to 100,000 inhabitants. This is the only way to compare areas with different population numbers.

The HZ is used as an indicator to express the threat caused by crime to an area. However, it should be noted that the HZ refers exclusively to the (reported) inhabitants of an area; other persons who frequent the area, e.g. customers, commuters or tourists, are not taken into account at the HZ. However, the crimes committed by this group of people are included in the calculation of the HZ. This means that the frequency figure for particularly attractive areas would actually be lower if the number of people exceeding the number of inhabitants could be quantified and included in the calculation. But that is not possible. High frequency figures are therefore not automatically a sign of negative living and quality of life, they can also be an expression of particular liveliness and popularity of areas.

In general, it is pointed out that the frequency figure alone should not be used without restriction for evaluation. This applies in particular to low-population district regions and to offences with basically low numbers of cases. Further explanations can be found at the top right under the button "Notes/Explanations".

In [26]:
# Load all sheets into a dictionary of DataFrames
excel_data = pd.read_excel(CRIME_DATA_PATH, skiprows=4, sheet_name=None)

# Display available sheet names
print(excel_data.keys())

# Access a specific sheet (e.g., 'Sheet1') or loop through all sheets if necessary
df = excel_data['HZ_2022']  # replace 'Sheet1' with the actual sheet name

# Preview the DataFrame
df.head()

# For mapping, select relevant columns
# Assuming 'LOR-Schlüssel (Bezirksregion)' as the district code and 'Straftaten -insgesamt-' as total crimes
# df = df[['LOR-Schlüssel (Bezirksregion)', 'Bezeichnung (Bezirksregion)', 'Straftaten -insgesamt-']]

# Preview the DataFrame after selection
# print(df.head())


dict_keys(['Titel', 'Inhaltsverzeichnis', 'Fallzahlen_2013', 'Fallzahlen_2014', 'Fallzahlen_2015', 'Fallzahlen_2016', 'Fallzahlen_2017', 'Fallzahlen_2018', 'Fallzahlen_2019', 'Fallzahlen_2020', 'Fallzahlen_2021', 'Fallzahlen_2022', 'HZ_2013', 'HZ_2014', 'HZ_2015', 'HZ_2016', 'HZ_2017', 'HZ_2018', 'HZ_2019', 'HZ_2020', 'HZ_2021', 'HZ_2022'])


,LOR-Schlüssel (Bezirksregion),Bezeichnung (Bezirksregion),Straftaten \n-insgesamt-,Raub,"Straßenraub,\nHandtaschen-raub",Körper-verletzungen \n-insgesamt-,Gefährl. und schwere Körper-verletzung,"Freiheits-beraubung, Nötigung,\nBedrohung, Nachstellung",Diebstahl \n-insgesamt-,Diebstahl von Kraftwagen,Diebstahl \nan/aus Kfz,Fahrrad-\ndiebstahl,Wohnraum-\neinbruch,Branddelikte \n-insgesamt-,Brand-\nstiftung,Sach-beschädigung -insgesamt-,Sach-beschädigung durch Graffiti,Rauschgift-delikte,Kieztaten
0,10000,Mitte,20717.605294,210.039532,128.116458,1969.471532,598.727513,790.391776,8932.167184,105.402584,1042.286088,1179.59018,147.512576,99.532707,28.583752,1574.40325,304.978422,846.538431,4556.045846
1,11001,Tiergarten Süd,25890.72139,232.90948,169.960972,2612.363087,881.279114,956.817323,9908.095178,107.012464,1227.495908,1926.224348,176.255823,169.960972,44.063956,2108.775022,302.152839,1428.931134,5627.596626
2,11002,Regierungsviertel,45667.738165,299.532437,153.419053,3609.000584,1081.239041,1198.129749,20521.624781,80.362361,1256.575102,1965.225015,233.781414,116.890707,29.222677,3177.966102,883.985973,1293.103448,8233.489188
3,11003,Alexanderplatz,31272.015011,268.295092,162.952234,3007.209402,842.742865,933.271883,14881.324686,82.299108,1017.216973,1402.376798,177.766073,113.572769,21.397768,2208.908055,528.360273,1139.019653,6259.670145
4,11004,Brunnenstraße Süd,11762.681103,200.269112,147.072629,979.441124,334.824921,388.021404,5444.816472,103.263761,660.262227,1320.524455,131.426604,43.808868,9.387615,1408.142191,312.920487,303.532872,2985.261445


# Plotting everything together

In [ ]:
# plot chloropleth map with districts and the crime rate
fig = px.choropleth_mapbox(
    df_polygon_postcode,
    geojson=berlin_district_polygon_geojson,
    locations='plz',
    color='plz',  # The data column you want to use for color
    featureidkey="properties.plz",  # Matches GeoJSON's postcode field
    title="Berlin Postcode Districts with Crime Rate",
   #  mapbox_style="carto-positron", # more monotone
    mapbox_style="open-street-map", # more vibrant
    center={"lat": 52.52, "lon": 13.405},  # Center on Berlin
    zoom=10,
    opacity=0.6,
)

# add line plot of the berlin wall data 
fig.add_trace(go.Scattermapbox(
    lat=berlinwall_df['lat'],
    lon=berlinwall_df['lon'],
    mode='lines',
    line=dict(color='black', width=3),  # Set line color and width
    name='Berlin Wall Boundary'
))

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={"x": 0.5, "xanchor": "center"},
)

fig.show()

In [60]:
# fig = go.Figure(data=go.Choropleth(
#     locations = country_df['country_code03'],
#     z = country_df['num_p_country'],
#     colorscale = 'YlGnBu',
#     autocolorscale=False,
#     reversescale=False,
#     marker_line_color='darkgray',
#     marker_line_width=0.5,
#     colorbar_tickprefix = '',
#     colorbar_title = 'No. Products',
# ))

# fig.update_layout(
#     title_text='No. Products Per Country that BCL Sells',
#     geo=dict(
#         showframe=True,
#         showcoastlines=True,
#         projection_type='equirectangular'
#     ),
#     width=800,
#     height=500,
#     annotations = [dict(
#         x=0.55,
#         y=-0.05,
#         xref='paper',
#         yref='paper',
#         text='Data: <a href="https://www.bcliquorstores.com">\
#             BCLIQUOR</a>',
#         showarrow = False
#     ),
#     dict(
#         x=0.84,
#         y=-0.1,
#         xref='paper',
#         yref='paper',
#         text='Analysis: <a href="https://github.com/johann997/BCL_WebScrape">\
#             github.com/johann997</a>',
#         showarrow = False
#     )
#     ]
# )


# fig.show()
# fig.write_html("figures/BCL-world-map.html")

## Useful Links for plotting these graphs

https://towardsdatascience.com/work-with-geospatial-data-and-create-interactive-maps-using-geopy-and-plotly-28178d2868f1 </br>
https://plotly.com/python/choropleth-maps/ </br>
https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv </br>
https://www.infragistics.com/help/reveal/location-data-requirements </br>